<a href="https://colab.research.google.com/github/watanabe-gk/study_gpt/blob/main/%E4%BA%8B%E5%89%8D%E5%AD%A6%E7%BF%92_%E5%95%86%E5%93%81%E3%83%AC%E3%83%92%E3%82%99%E3%83%A5%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
# @title モジュールのインストール
!pip install -qq datasets==2.18
!pip install -qq mecab-python3==1.0.8
!pip install -qq unidic-lite
!git clone https://github.com/Michi-123/whiteGPT.git

fatal: destination path 'whiteGPT' already exists and is not an empty directory.


In [78]:
#@title Google Driveのマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
#@title ライブラリーのインポート
import math
import torch
import torch.nn as nn
import torch.optim as optim # 最適化モジュール
from matplotlib import pyplot as plt # グラフの描画
import MeCab # 形態素解析

GPUの使用を判定

In [80]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [81]:
#@title MeCabの利用 (日本語を形態素解析するプログラム)
tagger = MeCab.Tagger("-Owakati")

In [82]:
#@title 青空文庫コーパスの利用
from datasets import load_dataset
ds = load_dataset('globis-university/aozorabunko-clean')
ds = ds.filter(lambda row: row['meta']['文字遣い種別'] == '新字新仮名')  # 新字新仮名に限定

In [83]:
authors = ['夏目','江戸川']
titles = ['こころ','魔術師']

In [84]:
#@title 青空文庫コーパスの読み込み
aozora_corpus = ''
for book in ds['train']:
    title = book['meta']['作品名']
    author = book['meta']['姓']
    if author in authors and title in titles:
        print(author, title)

        text = book['text'] # 本文
        text = ''.join(text.split()) # Clean up
        text = tagger.parse(text) # 形態素解析
        aozora_corpus += text

江戸川 魔術師
夏目 こころ


In [85]:
aozora_corpus[:500]

'作者 の 言葉 わが 明智 小五郎 は 、 遂に 彼 の 生涯 で の 最大 強敵 に 相対 し た 。 ここ に 『 蜘蛛 男 』 の 理智 を 越え て 変幻 自在 なる 魔術 が ある 。 魔術 師 は 看客 の 目 の 前 で 生き た 女 を 胴切り に し たり 、 箱詰め の 小女 を 剣 の 芋刺し に し たり 、 彼女 を 殺害 し て 鮮血 したたる 生首 を 転がし て 見せ たり 、 或は 立 所 に 人 を 眠ら せ 、 自由 自在 の 暗示 を 与え 、 或は 他人 の 心中 持物 を 看破 する など 、 あらゆる 奇怪 事 を 行う こと が 出来る 。 兇賊 が これ ら の 怪 技 の 妙 奥 を 会得 し て い た 場合 を 想像 せよ 。 流石 の 名 探偵 明智 小五郎 も この 魔術 師 の 心理 的 或は 物理 的 欺瞞 に は 、 いたく 悩まさ れ ね ば なら なかっ た 。 魔術 兇賊 と は 何者 で ある か 。 それ が どんな に 意外 な 人物 で ある か 。 又 彼 は そもそも 如何なる 悪業 を 企ん だ の '

# Hyper parameter

In [86]:
# @markdown Pre-trained model parameters

# 512 token GPT1
# 10
context_size = 15 # @param {type: "integer"}

# 768 GPT1 (512+256)
# 192
d_model = 128 #@param {type: "integer"}

# 12 GPT1
# 3
n_head = 2 #@param {type: "integer"}

# 12 layer in  GPT1
# 6
n_block = 3 #@param {type: "integer"}

# @markdown Train parameters
# 64 GPT1
# 64
batch_size = 64  #@param {type: "integer"}

# 2.5e-4
# 0.00025
learning_rate = 0.00025  #@param {type: "number"}

# 100 GPT1
num_epochs = 20 #@param {type: "integer"}

# pre-trained.pkl
pre_trained_model_path = '/content/drive/MyDrive/Study_GPT/models/pre-trained_item_review.pkl' #@param {type: "string"}
vocab_path = '/content/drive/MyDrive/Study_GPT/models/vocab_item_review.pkl' #@param {type: "string"}

In [87]:
# ハイパーパラメータの検証
if d_model % n_head != 0:
    print("埋め込みベクトルの次元はヘッドの数で割り切れる必要があります")
    raise ValueError("d_model must be divisible by n_head")

# ファイルの読み込み

In [88]:
# 基本文
corpus1 = open("/content/whiteGPT/corpus/pre-train/jp/basic.txt").read()
corpus1 = tagger.parse(corpus1)
corpus1[:100]

'これ は 、 自作 の " GPT " モデル の 事前 学習 用 に " Udemy " の 教材 と し て 用意 し まし た 。 ミカン と は 何 です か \u3000 ミカン は 、 果物 です '

In [89]:
# 商品レビュー
corpus2 = open("/content/whiteGPT/corpus/pre-train/jp/item_review.txt").read()
corpus2 = tagger.parse(corpus2)
corpus2[:100]

'化粧 品 の 購入 後 、 肌 が 明るく なり 、 自信 が 持てる よう に なり まし た 。 この テレビ は 不 具合 が 頻発 し 、 修理 に 出す こと が 多い 。 最新 の 冷蔵 '

In [90]:
# 基本文　＋　商品レビュー　＋　青空文庫
pre_train_corpus = corpus1 + corpus2 + aozora_corpus
pre_train_corpus[:100]

'これ は 、 自作 の " GPT " モデル の 事前 学習 用 に " Udemy " の 教材 と し て 用意 し まし た 。 ミカン と は 何 です か \u3000 ミカン は 、 果物 です '

In [91]:
# ファインチューニングで使うコーパス　（※学習には使わない）
fine_tuning_corpus = open("/content/whiteGPT/corpus/fine-tuning/text_classification.txt").read()
fine_tuning_corpus = tagger.parse(fine_tuning_corpus)
fine_tuning_corpus[:100]

'この コーヒー メーカー は コーヒー の 味 が 本格 的 で 、 毎朝 の 楽しみ です 。 大 満足 です 。 : 肯定 新しい 目覚まし 時計 を 買い まし た 。 この 製品 は 、 あま'

# インスタンス化

In [92]:
from whiteGPT import GPT, GPT2, create_attention_mask # 完成コードの利用
from whiteGPT import Evaluate, Vocab
from whiteGPT import DataLoader, TextDataset as Dataset

In [93]:
# 単語辞書
vocab = Vocab(pre_train_corpus + fine_tuning_corpus)
vocab.index2word

{0: '<PAD>',
 1: '<BOS>',
 2: '<EOS>',
 3: '<UNK>',
 4: '<EXT1>',
 5: '<EXT2>',
 6: '!',
 7: '"',
 8: '#',
 9: '(',
 10: ')',
 11: '*',
 12: '+',
 13: ',',
 14: '-',
 15: '--',
 16: '.',
 17: '/',
 18: '0',
 19: '1',
 20: '10',
 21: '1000',
 22: '12',
 23: '120',
 24: '13',
 25: '134',
 26: '15',
 27: '170',
 28: '1774',
 29: '18',
 30: '1808',
 31: '1832',
 32: '19',
 33: '1900',
 34: '1950',
 35: '1960',
 36: '1972',
 37: '1980',
 38: '1990',
 39: '1996',
 40: '2',
 41: '20',
 42: '200',
 43: '2000',
 44: '2007',
 45: '2024',
 46: '21',
 47: '25',
 48: '280',
 49: '3',
 50: '30',
 51: '360',
 52: '3776',
 53: '4',
 54: '40',
 55: '400',
 56: '480',
 57: '5',
 58: '50',
 59: '575',
 60: '6',
 61: '600',
 62: '64',
 63: '65',
 64: '7',
 65: '8',
 66: '9',
 67: ':',
 68: '?',
 69: 'A',
 70: 'AI',
 71: 'AR',
 72: 'ARPANET',
 73: 'Amazon',
 74: 'Android',
 75: 'Apple',
 76: 'Assistant',
 77: 'B',
 78: 'Bluetooth',
 79: 'Bus',
 80: 'C',
 81: 'CFC',
 82: 'CO',
 83: 'CPU',
 84: 'ChatGPT',
 8

In [94]:
# Dataset と DataLoader
dataset = Dataset(vocab, pre_train_corpus, context_size)
train_dataloader = DataLoader(dataset, batch_size, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

In [95]:
# 語彙数の取得
vocab_size = dataset.vocab_size
vocab_size

14597

In [96]:
dataset.word2index['<PAD>']


0

In [97]:
vocab.word2index['私']

10878

In [98]:
# 語彙の辞書をファイルに保存
import pickle
with open(vocab_path, 'wb') as f:
    pickle.dump(vocab, f)

In [99]:
# マスク関数
mask = create_attention_mask(context_size)
mask.to(device)

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')

In [100]:
# 評価モジュールの利用
evaluate = Evaluate(dataset, context_size)

In [101]:
# GPTモデルの作成とデバイス対応
model = GPT2(vocab_size, context_size, d_model, n_head, n_block)
model.to(device)

GPT2(
  (token_embedding): Embedding(14597, 128)
  (positional_encoding): PositionalEncoding()
  (dropout): Dropout(p=0.1, inplace=False)
  (transformer_block): ModuleList(
    (0-2): 3 x TransformerBlock(
      (norm_1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (norm_2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (attn): MultiHeadAttention(
        (fc_q): Linear(in_features=128, out_features=128, bias=True)
        (fc_k): Linear(in_features=128, out_features=128, bias=True)
        (fc_v): Linear(in_features=128, out_features=128, bias=True)
        (attention): ScaledDotProductAttention(
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (fc): Linear(in_features=128, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (fc1): Linear(in_features=128, out_features=512, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (fc2): Linear(in_features=512,

In [102]:
# クロスエントロピー誤差
criterion = nn.CrossEntropyLoss()
# 最適化モジュールの利用
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [103]:
is_load_model = False # @param{type:"boolean"}
if is_load_model:
    model.load_state_dict(torch.load(pre_trained_model_path, map_location=device))
    print('ok')

# 学習

In [104]:
#@title 初期化
start_epoch = 0
history = {"train_loss": []}

In [105]:
start_epoch = 0  # から続き

In [106]:
from tqdm import tqdm

In [107]:
#@title Train
model = model.to(device)
mask = mask.to(device)

# Epoch数まで繰り返し
for epoch in range(start_epoch, num_epochs):

    # 損失の累計をリセット
    running_loss = 0.0

    # 学習モードに変更
    model.train()

    # 学習データセット全体をバッチ単位で処理
    for i, batch in enumerate(tqdm(train_dataloader)):

        # 入力元データと教師データ
        source = batch['source'].to(device)
        target = batch['target'].to(device).view(-1)

        # 勾配のリセット
        optimizer.zero_grad()

        # 推論
        outputs, _, _ = model(source, None, mask)

        # 損失計算
        loss = criterion(outputs, target)

        # 誤差逆伝播
        loss.backward()

        # 学習パラメータの更新
        optimizer.step()

        # 損失の累積
        running_loss += loss.item()

    # 損失の履歴を作成
    train_loss = running_loss / len(train_dataloader)
    history["train_loss"].append(train_loss)

    # 開始epochの更新
    start_epoch = epoch

    # 損失の確認
    if epoch % 1 == 0:
        print(f"epoch:{epoch+1}  index:{i+1} loss:{train_loss:.6f}")
        evaluate.predict(source, target, outputs)

    # モデルの保存
    if epoch % 1 == 0 and epoch != 0:
        torch.save(model.state_dict(), pre_trained_model_path )
        print("Saved.")

    # グラフの描画
    if epoch % 5 == 0 and epoch != 0:
        plt.plot(history['train_loss'])
        plt.show()


100%|██████████| 4585/4585 [01:37<00:00, 47.05it/s]


epoch:1  index:4585 loss:5.585066
source 連中 も 、 娘 の 文代 まで 、 行衛 知れ ず に なっ て い 
target た
output た



  4%|▍         | 176/4585 [00:04<01:44, 42.29it/s]


KeyboardInterrupt: 

# Test

In [ ]:
sample_corpus = "この商品はおすすめできますが、温度調整が不安定で" #@param{type:"string"}
parsed_corpus = tagger.parse(sample_corpus)
seq_size = len(parsed_corpus.split())
mask = create_attention_mask(context_size )
# mask[:,seq_size:] = 0
Mask = None
print(parsed_corpus.split()[:context_size])
evaluate.generate2(parsed_corpus, model, mask)